In [5]:
import scipy
from scipy import fftpack, signal
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def pyr2im(pyr):
    pass

def im2pyr(im, d, n, k, filter):
    pass

def frames2video(frames):
    pass

def modify_motion(video, alpha, D, N, K, filter1, filter2):
    # process video to frames, compute frame_rate
    frames = []
    frame_rate = 0

    # obtain steerable pyramid of shape (d, n, k)
    pyr = [[[[] for i in range(K)] for i in range(N)] for i in range(D)]
    for t, frame in enumerate(frames):
        pyr[t] = pyr2im(frame, D, N, K, filter1)

    # modify motion
    modified_pyr = [[[[] for i in range(K)] for i in range(N)] for i in range(D)]
    for d in range(D):
        for n in range(N):
            for k in range(K):
                modified_pyr[:, d, n, k] = perform_temporal_modification(pyr[:, d, n, k], alpha, filter2)

    # collapse pyramid
    modified_frames = []
    for t in range(len(pyr)):
        modified_frames[t] = pyr2im(pyr[t])

    # output video
    output_video = frames2video(modified_frames)
    return output_video

In [161]:
def temporal_filtering(I, F):
    _, h, w = I.shape
    J = np.empty(shape=I.shape, dtype=np.complex)
    for i in range(w*h):
        row, col = np.unravel_index(i, (h, w))
        x = I[:, row, col]
        x_dft = fftpack.fft(x)        
        y_dft = x_dft * F
        J[:, row, col] = fftpack.ifft(y_dft)
    return J

In [163]:
def modify_motion(alpha, delta_phi, I):
    return I * np.exp((alpha - 1) * np.complex(0, 1) * delta_phi)

In [181]:
def perform_temporal_modification(frames, alpha, temporal_filter):
   delta_phi = temporal_filtering(np.angle(frames), temporal_filter)
   
   J = np.empty(shape=frames.shape)
   for i in range(delta_phi.shape[0]):
       J[i] = modify_motion(alpha, delta_phi[i], frames[i])

In [165]:
F = scipy.fftpack.fft(scipy.fftpack.ifftshift(scipy.signal.firwin(50,[1,3],fs=9,pass_zero=False)))

delta_phi = temporal_filtering(np.angle(I1), F)

J = np.empty(shape=I1.shape)
for i in range(delta_phi.shape[0]):
    J[i] = modify_motion(10, delta_phi[i], I1[i])

/Users/SamYang95/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  import sys


In [86]:
I1 = np.ones(shape=(50, 9, 10))
idx, inc = 0, -1

I1[0, 4, 0] = 0
for i, img in enumerate(I1[1:]):
    if i % 9 == 0: inc *= -1  
    idx += inc  
    img[4][idx] = 0